## Remove punctuation marks from words

In [ ]:
import re

def remove_spl_char(sent):
    
    punctuation = re.compile('[^a-zA-Z\-]*')
    post_punctutation = []
    for word in sent:
        word=punctuation.sub("", word.lower())
        if len(word) > 0 and word != 's':
            post_punctutation.append(word)

    return post_punctutation

## Read corpus files

In [ ]:
from nltk import corpus
list_of_sent = []
for name in corpus.gutenberg.fileids():
    sublist_of_sent = corpus.gutenberg.sents(name)
    list_of_sent.extend(sublist_of_sent)

## tokenize sentences

In [ ]:
list_of_tokenized_sent = [remove_spl_char(sent) for sent in list_of_sent]
# list_of_tokenized_sent
for elem in list_of_tokenized_sent:
    if len(elem) <= 3:
        list_of_tokenized_sent.remove(elem)

## Apply word2vec

In [ ]:
import gensim
ml=gensim.models.Word2Vec(list_of_tokenized_sent, size=50, min_count=1)

## Prepare input and target arrays

In [ ]:
import pandas as pd
word_input = [ [ ml.wv[sent[i]], ml.wv[sent[i+1]], ml.wv[sent[i+2]] ] for sent in list_of_tokenized_sent for i in range(len(sent)-3)]

In [ ]:
word_output = [ ml.wv[sent[i]] for sent in list_of_tokenized_sent for i in range(3, len(sent))]

## Import modules

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
import numpy as np

## split train and test data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(word_input), np.array(word_output), test_size=0.2, random_state =22)

## Construct LSTM

In [ ]:
model = Sequential()
model.add(LSTM((350), activation = 'relu', batch_input_shape=(None, 3, 50), return_sequences = True))
model.add(LSTM((150), activation = 'relu', return_sequences = True))
model.add(LSTM((50), activation = 'relu', return_sequences = False))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

model.summary()

## Train with data

In [ ]:
history = model.fit(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

## Analysis

In [ ]:
dictionary=dict(zip(ml.wv.vectors[0], ml.wv.index2word))
dictionary[ml['the'][0]]
y_test[:, 0]
model.predict(x_test)[:, 0]